In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def MNIST_load():
    from keras.datasets import mnist
    (X, y), (_, _ )= mnist.load_data()
    X = X.reshape(-1, 784)
    X = X.astype(np.float32) / 255.0
    y = y.astype(np.int32)
    return X, y

In [3]:
def one_hot_encode(y):
    num_classes = np.unique(y).shape[0]
    y_one_hot = np.zeros((y.shape[0], num_classes), dtype=int)
    y_one_hot[np.arange(y.shape[0]), y] = 1
    return y_one_hot

In [4]:
X, y = MNIST_load()

2024-09-25 15:56:47.476872: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 15:56:47.476899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 15:56:47.477831: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 15:56:47.484372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 15:56:48.174639: W tensorflow/compiler/tf2

In [5]:
def train_test_split(X, y, split_ratio = 0.7):
    N = X.shape[0]
    rnd_idx = np.random.permutation(N)
    train_idx = rnd_idx[:int(split_ratio * X.shape[0])]
    test_idx = rnd_idx[int(split_ratio * X.shape[0]):]
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_test = X[test_idx]
    y_test = y[test_idx]
    return X_train, y_train, X_test, y_test

In [6]:
X_train, y_train, X_test, y_test = train_test_split(X, y)
y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

In [7]:
def ReLU(x):
    return np.maximum(0, x)

In [11]:
class ELM:
    def __init__(self, X, y, hidden_dim):
        self.X = X #X_train
        self.y = y #y_train (one_hot_encoded)
        self.input_dim = self.X.shape[1]
        self.hidden_dim = hidden_dim
        self.W = np.random.rand(self.input_dim, self.hidden_dim)
        
    def input_hidden(self, X): #compute output of input -> hidden layer
        return ReLU(np.dot(X, self.W))
    
    def train(self):
        X_ = self.input_hidden(self.X)
        self.β = np.dot(np.linalg.pinv(X_), self.y)
    
    def predict(self, X):
        X_ = self.input_hidden(X)
        X_ = np.dot(X_, self.β)
        pred = np.argmax(X_, axis = 1)
        return pred

    def get_accuracy(self, X_train, X_test, y_train, y_test):
        # y (non one_hot_encoded)
        train_pred = self.predict(X_train)
        test_pred = self.predict(X_test)
        train_acc = np.sum(train_pred == y_train)/X_train.shape[0]
        test_acc = np.sum(test_pred == y_test)/X_test.shape[0]
        return train_acc, test_acc

In [12]:
net = ELM(X_train, y_train_one_hot, 200)
net.train()
train_acc, test_acc = net.get_accuracy(X_train, X_test, y_train, y_test)

In [15]:
def run(n_sims, X, y, save = False):
    # Train and test accuracies
    train_accs = []
    test_accs = []
    
    #hidden dimension layer range
    hid_dim_range = np.arange(10, 200, 10)
    for hid_dim in hid_dim_range:
        # train and test accs for a specific hid_dim
        train_accuracy = [] 
        test_accuracy = []
        print(f"With hidden dimension {hid_dim}.")
        for sim in range(n_sims):
            print(f"Simulation {sim + 1}/{n_sims}")
            X_train, y_train, X_test, y_test = train_test_split(X, y)
            y_train_one_hot = one_hot_encode(y_train)
            y_test_one_hot = one_hot_encode(y_test)

            net = ELM(X_train, y_train_one_hot, hid_dim) #init
            net.train() #train
            train_acc, test_acc = net.get_accuracy(X_train, X_test, y_train, y_test)
            train_accuracy.append(train_acc)
            test_accuracy.append(test_acc)
            
        train_accs.append((np.mean(train_accuracy), np.std(train_accuracy)))
        test_accs.append((np.mean(test_accuracy), np.std(test_accuracy)))
    train_μ, train_σ = zip(*train_accs)
    test_μ, test_σ = zip(*test_accs)

    plt.errorbar(hid_dim_range, train_μ, yerr=train_σ, fmt=".", capsize=2, capthick=1, ecolor="r", label="Train acc")
    plt.errorbar(hid_dim_range, test_μ, yerr=test_σ, fmt=".", capsize=2, capthick=1, ecolor="k", label="Test acc")
    plt.xlabel("Hidden dim")
    plt.ylabel("Accuracy")
    plt.legend()

    if save:
        plt.savefig("performance", format="png", dpi=300)
    
    plt.show()

In [16]:
run(10, X, y, save=True)

With hidden dimnsion 10.
Simulation 1/10
Simulation 2/10
Simulation 3/10
Simulation 4/10
Simulation 5/10
Simulation 6/10
Simulation 7/10
Simulation 8/10
Simulation 9/10
Simulation 10/10
With hidden dimnsion 20.
Simulation 1/10
Simulation 2/10
Simulation 3/10
Simulation 4/10
Simulation 5/10
Simulation 6/10
Simulation 7/10
Simulation 8/10
Simulation 9/10
Simulation 10/10
With hidden dimnsion 30.
Simulation 1/10
Simulation 2/10
Simulation 3/10
Simulation 4/10
Simulation 5/10
Simulation 6/10
Simulation 7/10
Simulation 8/10
Simulation 9/10
Simulation 10/10
With hidden dimnsion 40.
Simulation 1/10
Simulation 2/10
Simulation 3/10
Simulation 4/10
Simulation 5/10
Simulation 6/10
Simulation 7/10
Simulation 8/10
Simulation 9/10
Simulation 10/10
With hidden dimnsion 50.
Simulation 1/10
Simulation 2/10
Simulation 3/10
Simulation 4/10
Simulation 5/10
Simulation 6/10
Simulation 7/10


KeyboardInterrupt: 